In [1]:
%pip install icechunk "xarray>=2024.11.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.7 MB/s eta 0:00:00
  Attempting uninstall: numcodecs
    Found existing installation: numcodecs 0.12.1
    Uninstalling numcodecs-0.12.1:
      Successfully uninstalled numcodecs-0.12.1
  Attempting uninstall: xarray
    Found existing installation: xarray 2024.7.0
    Uninstalling xarray-2024.7.0:
      Successfully uninstalled xarray-2024.7.0
  Attempting uninstall: zarr
    Found existing installation: zarr 2.18.2
    Uninstalling zarr-2.18.2:
      Successfully uninstalled zarr-2.18.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmip 0.7.2 requires xgcm<0.7.0, but you have xgcm 0.8.1 which is incompatible.
Note: you may need to restart th

In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import earthaccess
import odc.stac
import pyproj
import pystac
import pystac_client
import xarray as xr
from dask.diagnostics import ProgressBar
from icechunk import Repository, Storage
from rasterio.enums import Resampling
from shapely.geometry import box
from shapely.ops import transform

In [2]:
CRS_STRING = "epsg:5070"
EPSG = pyproj.CRS.from_string(CRS_STRING).to_epsg()

START_DATE = datetime(year=2024, month=1, day=1)
END_DATE = datetime(year=2024, month=12, day=31, hour=23)
HLS_NODATA = -9999
HLS_COLLECTION_IDS = ["HLSL30_2.0", "HLSS30_2.0"]

URL_PREFIX = "https://data.lpdaac.earthdatacloud.nasa.gov/"

def convert_to_s3_uri(url: str) -> str:
    return url.replace(
        URL_PREFIX,
        "s3://"
    )

# 100 km bounding box
xmin, ymin = 150000, 2650000
bbox = (xmin, ymin, xmin + 200000, ymin + 200000)
aoi = box(*bbox)

# STAC items store bounding box info in epsg:4326
transformer_4326 = pyproj.Transformer.from_crs(
    crs_from=CRS_STRING,
    crs_to="epsg:4326",
    always_xy=True,
)

bbox_4326 = transform(transformer_4326.transform, aoi).bounds

In [3]:
auth = earthaccess.login(strategy="interactive", persist=True)
s3_creds = auth.get_s3_credentials("LPDAAC")

odc.stac.configure_rio(
    cloud_defaults=True,
    aws=dict(
        aws_access_key_id=s3_creds["accessKeyId"],
        aws_secret_access_key=s3_creds["secretAccessKey"],
        aws_session_token=s3_creds["sessionToken"],
        region_name="us-west-2",
    ),
)

In [4]:
item_collection_json = "hls_items.json"

load_from_json = True

if load_from_json:
    stac_items = pystac.ItemCollection.from_file(item_collection_json)
else:
    catalog = pystac_client.Client.open("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")
    
    all_items = []
    current = START_DATE
    while current <= END_DATE:
        print(current)
        # Calculate the end of the current month
        # Move to first of next month and subtract one second
        next_month = (current + relativedelta(months=1))
        month_end = next_month - timedelta(seconds=1)
        
        # If this is the last interval and it exceeds end_date,
        # use end_date as the interval end
        if month_end > END_DATE:
            month_end = END_DATE
            
        monthly_stac_items = catalog.search(
            collections=HLS_COLLECTION_IDS,
            bbox=bbox_4326,
            datetime=[current, month_end],
            limit=1000,
        ).item_collection()
    
        all_items.extend(monthly_stac_items.items)
        
        current = next_month
    
    stac_items = pystac.ItemCollection(all_items)
    
    stac_items.save_object(item_collection_json)

print(len(stac_items))

1750


In [14]:
for item in stac_items:
    for asset in item.assets.values():
        if asset.href.startswith(URL_PREFIX):
            asset.href = convert_to_s3_uri(asset.href)

def group_by_sensor_and_date(
    item: pystac.Item,
    parsed: odc.stac.ParsedItem,
    idx: int,
) -> str:
    id_split = item.id.split(".")
    sensor = id_split[1]
    day = id_split[3][:7]

    return f"{sensor}_{day}"

HLS_ODC_STAC_CONFIG = {
    "HLSL30_2.0": {
        "assets": {
            "*": {
                "nodata": HLS_NODATA,
                "data_type": "int16",
            },
            "Fmask": {
                "nodata": 0,
                "data_type": "uint8",
            },
        },
        "aliases": {
            "coastal_aerosol": "B01",
            "blue": "B02",
            "green": "B03",
            "red": "B04",
            "nir_narrow": "B05",
            "swir_1": "B06",
            "swir_2": "B07",
            "cirrus": "B09",
            "thermal_infrared_1": "B10",
            "thermal": "B11",
        },
    },
    "HLSS30_2.0": {
        "assets": {
            "*": {
                "nodata": HLS_NODATA,
                "dtype": "int16",
            },
            "Fmask": {
                "nodata": 0,
                "dtype": "uint8",
            },
        },
        "aliases": {
            "coastal_aerosol": "B01",
            "blue": "B02",
            "green": "B03",
            "red": "B04",
            "red_edge_1": "B05",
            "red_edge_2": "B06",
            "red_edge_3": "B07",
            "nir_broad": "B08",
            "nir_narrow": "B8A",
            "water_vapor": "B09",
            "cirrus": "B10",
            "swir_1": "B11",
            "swir_2": "B12",
        },
    },
}

# these are the ones that we are going to use
BANDS = ["red", "green", "blue", "Fmask"]

stack = odc.stac.load(
    stac_items,
    stac_cfg=HLS_ODC_STAC_CONFIG,
    bands=BANDS,
    crs=CRS_STRING,
    resolution=30,
    chunks={"time": 1, "x": 500, "y": 500},
    groupby=group_by_sensor_and_date,
    x=(bbox[0], bbox[2]),
    y=(bbox[1], bbox[3]),
    patch_url=convert_to_s3_uri,
).sortby("time")

display(stack)

<xarray.Dataset> Size: 107GB
Dimensions:      (y: 6667, x: 6667, time: 344)
Coordinates:
  * y            (y) float64 53kB 2.85e+06 2.85e+06 ... 2.65e+06 2.65e+06
  * x            (x) float64 53kB 1.5e+05 1.5e+05 1.501e+05 ... 3.5e+05 3.5e+05
    spatial_ref  int32 4B 5070
  * time         (time) datetime64[ns] 3kB 2024-01-01T17:04:41.320000 ... 202...
Data variables:
    red          (time, y, x) int16 31GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    green        (time, y, x) int16 31GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    blue         (time, y, x) int16 31GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    Fmask        (time, y, x) uint8 15GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>

In [15]:
hls_mask_bitfields = [1, 2, 3]  # cloud shadow, adjacent to cloud shadow, cloud
hls_bitmask = 0
for field in hls_mask_bitfields:
    hls_bitmask |= 1 << field

fmask = stack["Fmask"].astype("uint16")
mask = fmask & hls_bitmask

nodata_mask = stack["red"] == HLS_NODATA

cloud_free = stack.where(mask == 0).where(nodata_mask == 0)

monthly_cloud_free = cloud_free[["red", "green", "blue"]].resample(time="ME").median(skipna=True)
monthly_cloud_free

<xarray.Dataset> Size: 6GB
Dimensions:      (time: 12, y: 6667, x: 6667)
Coordinates:
  * y            (y) float64 53kB 2.85e+06 2.85e+06 ... 2.65e+06 2.65e+06
  * x            (x) float64 53kB 1.5e+05 1.5e+05 1.501e+05 ... 3.5e+05 3.5e+05
    spatial_ref  int32 4B 5070
  * time         (time) datetime64[ns] 96B 2024-01-31 2024-02-29 ... 2024-12-31
Data variables:
    red          (time, y, x) float32 2GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    green        (time, y, x) float32 2GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>
    blue         (time, y, x) float32 2GB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>

In [16]:
storage_config = Storage.new_local_filesystem("icechunk-test-30m")
repo = Repository.open_or_create(storage_config)

In [ ]:
for i, t in enumerate(monthly_cloud_free.time.values):
    print(t)
    session = repo.writable_session("main")

    ds = monthly_cloud_free.sel(time=slice(t, t))
    ds.to_zarr(session.store, zarr_format=3, consolidated=False, append_dim="time" if i > 0 else None)
    session.commit(f"add {t} data to store")

2024-01-31T00:00:00.000000000


/srv/conda/envs/notebook/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


2024-02-29T00:00:00.000000000
